In [1]:
# %pip install cloudscraper

import pandas as pd
from bs4 import BeautifulSoup, Comment
import re
import cloudscraper
import time
import random
import csv

SEASON_LIST=['2021', '2022', '2023', '2024', '2025']
SEASON_URL = 'https://www.pro-football-reference.com/years/{}/games.htm'
GAME_URL = 'https://www.pro-football-reference.com/boxscores/{}.htm' # add game_id to the end of it
GAME_URL_LIST = []



In [ ]:
# Create a scraper that bypasses Cloudflare
scraper = cloudscraper.create_scraper(
    browser={
        'browser': 'chrome',
        'platform': 'windows',
        'mobile': False
    }
)

game_id_list = []

for year in SEASON_LIST:
    url = SEASON_URL.format(year)
    # print(f"Processing {year}...")
    
    # Add random delay
    time.sleep(random.uniform(2, 4))
    
    try:
        response = scraper.get(url)
        # print(f"Status code: {response.status_code}")
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            # print(f"Successfully fetched {url}")
            
            # Find the table with id="games"
            games_table = soup.find('table', {'id': 'games'})
            
            if games_table:
                # print(f"Found games table for {year}")

                # Method 1: Exact text match
                boxscore_links = games_table.find_all('a', string='boxscore')
                
                print(f"Found {len(boxscore_links)} boxscore links for {year}")
                
                # Extract href attributes
                for link in boxscore_links:
                    href = link.get('href')
                    if href:
                        # print(f"Found href: {href}")

                        # 1. Get the last part after last slash
                        game_id = href.split('/')[-1]
                        
                        # 2. Remove file extension if present
                        if '.' in game_id:
                            game_id = game_id.split('.')[0]
                            
                        
                        game_id_list.append(game_id)
                        # print(f"  → Extracted game ID: {game_id}")
        else:
            print(f"Failed with status: {response.status_code}")
            
    except Exception as e:
        print(f"Error: {e}")

# print(len(game_id_list))
# game_id_list

Found 285 boxscore links for 2021
Found 284 boxscore links for 2022
Found 285 boxscore links for 2023
Found 285 boxscore links for 2024
Found 243 boxscore links for 2025


['202109090tam',
 '202109120atl',
 '202109120buf',
 '202109120car',
 '202109120cin',
 '202109120clt',
 '202109120oti',
 '202109120det',
 '202109120htx',
 '202109120was',
 '202109120kan',
 '202109120nyg',
 '202109120nor',
 '202109120nwe',
 '202109120ram',
 '202109130rai',
 '202109160was',
 '202109190mia',
 '202109190car',
 '202109190chi',
 '202109190cle',
 '202109190clt',
 '202109190jax',
 '202109190nyj',
 '202109190phi',
 '202109190pit',
 '202109190tam',
 '202109190crd',
 '202109190sdg',
 '202109190sea',
 '202109190rav',
 '202109200gnb',
 '202109230htx',
 '202109260nyg',
 '202109260buf',
 '202109260cle',
 '202109260pit',
 '202109260oti',
 '202109260jax',
 '202109260det',
 '202109260kan',
 '202109260nwe',
 '202109260den',
 '202109260rai',
 '202109260min',
 '202109260ram',
 '202109260sfo',
 '202109270dal',
 '202109300cin',
 '202110030atl',
 '202110030buf',
 '202110030dal',
 '202110030chi',
 '202110030min',
 '202110030mia',
 '202110030phi',
 '202110030nor',
 '202110030nyj',
 '202110030ram

In [3]:
game_content = {}

def get_data(game_id_list):
   for game_id in game_id_list:

      failed_ids = []

      url = GAME_URL.format(game_id)
      print("Processing: " + url)

      # Add random delay
      time.sleep(random.uniform(2, 4))

      try:
         response = scraper.get(url)
         print(f"Status code: {response.status_code}")
         
         if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            print(f"Successfully fetched {url}")            

            ##### get general data #####
            scorebox_meta_div = soup.find('div', {'class': 'scorebox_meta'})
            if scorebox_meta_div:
               # print(f"Found scorebox_meta div for {game_id}")
               
               # Get all text content with line breaks
               full_text = scorebox_meta_div.get_text(separator='\n')
               
               # Initialize variables
               date = None
               stadium = None
               
               # Extract date - first line is usually the date
               lines = [line.strip() for line in full_text.split('\n') if line.strip()]
               
               date = lines[0]
               stadium = lines[5]      
                  
            away_scorebox_div = soup.find('div', {'id': 'sb_team_0'})
            if away_scorebox_div:
               # print(f"Found away_scorebox_div div for {game_id}")
               
               full_text = away_scorebox_div.get_text(separator='\n')
               
               lines = [line.strip() for line in full_text.split('\n') if line.strip()]

               away_team = lines[2]
               # print(away_team)

            home_scorebox_div = soup.find('div', {'id': 'sb_team_1'})
            if home_scorebox_div:
               # print(f"Found home_scorebox_div div for {game_id}")
               
               full_text = home_scorebox_div.get_text(separator='\n')
               
               lines = [line.strip() for line in full_text.split('\n') if line.strip()]

               home_team = lines[2]
               # print(home_team)

            # set the scorebox content
            scorebox_content = (home_team, away_team, date, stadium)

            ##### get environment data #####
            game_info_div = soup.find('div', {'id': 'all_game_info'})
            if game_info_div:
               # print(type(game_info_div))
                  
               # Look for the Roof, Surface, Weather lines in the comment
               roof_match = re.search(r'<th[^>]*>Roof</th><td[^>]*>(.*?)</td>', str(game_info_div))
               surface_match = re.search(r'<th[^>]*>Surface</th><td[^>]*>(.*?)</td>', str(game_info_div))
               weather_match = re.search(r'<th[^>]*>Weather</th><td[^>]*>(.*?)</td>', str(game_info_div))
               
               roof = roof_match.group(1) if roof_match else None
               surface = surface_match.group(1) if surface_match else None
               weather = weather_match.group(1) if weather_match else None

               if weather:
                  weather_content = weather.split(', ')

                  temp = weather_content[0][:2]

                  humidity = weather_content[1].split(' ')[-1]

                  wind = weather_content[2][-5:]
               else:
                  temp = None
                  humidity = None
                  wind = None

               environment_content = (roof, surface, temp, humidity, wind)
            

            ##### get snap data #####

            # get player snaps for each row in home and away tables
            def extract_player_snaps(row):
               
               # Get player name from the link in the first th
               player_cell = row.find('th', {'data-stat': 'player'})
               player_link = player_cell.find('a')
               player_name = player_link.text if player_link else player_cell.text.strip()
               
               # Extract snap counts using data-stat attributes
               off_snaps = int(row.find('td', {'data-stat': 'offense'}).text.strip())
               def_snaps = int(row.find('td', {'data-stat': 'defense'}).text.strip())
               st_snaps = int(row.find('td', {'data-stat': 'special_teams'}).text.strip())

               total_snaps = off_snaps + def_snaps + st_snaps

               return {player_name: total_snaps} 

            comments = soup.find_all(string=lambda text: isinstance(text, Comment)) # find it in the comments

            # get data for home team
            for comment in comments:
               if 'home_snap_counts' in comment:
                  # print("Found table in comment!")
                  
                  # Parse the comment content as HTML
                  comment_soup = BeautifulSoup(comment, 'html.parser')
                  
                  # Now find the table
                  home_snap_counts_table = comment_soup.find('table', id='home_snap_counts')
                  break

            home_tbody = home_snap_counts_table.find('tbody')
            if home_tbody:
               # print("Found home_tbody")

               home_player_snap_counts = []

               rows = home_tbody.find_all('tr')
               for row in rows:
                  # print(row)

                  player_snap_data = extract_player_snaps(row)
                  home_player_snap_counts.append(player_snap_data)

            # get data for away team
            for comment in comments:
               if 'vis_snap_counts' in comment:
                  # print("Found table in comment!")
                  
                  # Parse the comment content as HTML
                  comment_soup = BeautifulSoup(comment, 'html.parser')
                  
                  # Now find the table
                  away_snap_counts_table = comment_soup.find('table', id='vis_snap_counts')
                  break

            away_tbody = away_snap_counts_table.find('tbody')
            if away_tbody:
               # print("Found away_tbody")

               away_player_snap_counts = []

               rows = away_tbody.find_all('tr')
               for row in rows:
                  # print(row)

                  player_snap_data = extract_player_snaps(row)
                  away_player_snap_counts.append(player_snap_data)

         elif response.status_code == 429:
            # time.sleep(random.uniform(2, 4))
            print("429 error at " + game_id)
            game_id_429 = game_id
            break

         else:
            print(f"Failed with status: {response.status_code}")
            failed_ids.append(game_id)
               
      except Exception as e:
         print(f"Error: {e}")
         failed_ids.append(game_id)

      with open('player_snap_data.csv', 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)

            for player_dict in home_player_snap_counts:
            # player_dict looks like: {'Andrew Wylie': 75}
               for player_name, total_snaps in player_dict.items():
                     writer.writerow([
                        game_id,
                        player_name,  # Extract player name from dictionary key
                        date,
                        total_snaps,  # Extract total snaps from dictionary value
                        home_team, # which team they are on
                        home_team, # the teams involved
                        away_team,
                        True,  # is_home
                        stadium,
                        weather,
                        surface,
                        temp,
                        humidity,
                        wind,
                        roof
                     ])

            for player_dict in away_player_snap_counts:
            # player_dict looks like: {'Andrew Wylie': 75}
               for player_name, total_snaps in player_dict.items():
                     writer.writerow([
                        game_id,
                        player_name,  # Extract player name from dictionary key
                        date,
                        total_snaps,  # Extract total snaps from dictionary value
                        away_team, # which team they are on
                        home_team, # the teams involved
                        away_team,
                        False,  # is_home
                        stadium,
                        weather,
                        surface,
                        temp,
                        humidity,
                        wind,
                        roof
                     ])


      # game_content[(game_id, scorebox_content)] = (environment_content, home_player_snap_counts, away_player_snap_counts)

      # break

   

In [4]:
get_data(game_id_list)

# # if there is a 429 error, manually set game_id_429 to the correct game_id, comment out the above get_data function call
# game_id_429 = "202009100kan"
# game_content_start_ind = game_id_list.index(game_id_429)
# updated_game_id_list = game_id_list[game_content_start_ind:]

# get_data(updated_game_id_list)

# print(game_content)


Processing: https://www.pro-football-reference.com/boxscores/202109090tam.htm
Status code: 200
Successfully fetched https://www.pro-football-reference.com/boxscores/202109090tam.htm
Processing: https://www.pro-football-reference.com/boxscores/202109120atl.htm
Status code: 200
Successfully fetched https://www.pro-football-reference.com/boxscores/202109120atl.htm
Processing: https://www.pro-football-reference.com/boxscores/202109120buf.htm
Status code: 200
Successfully fetched https://www.pro-football-reference.com/boxscores/202109120buf.htm
Processing: https://www.pro-football-reference.com/boxscores/202109120car.htm
Status code: 200
Successfully fetched https://www.pro-football-reference.com/boxscores/202109120car.htm
Processing: https://www.pro-football-reference.com/boxscores/202109120cin.htm
Status code: 200
Successfully fetched https://www.pro-football-reference.com/boxscores/202109120cin.htm
Processing: https://www.pro-football-reference.com/boxscores/202109120clt.htm
Status code: